In [1]:
from numpy import *
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import glob
import alsvinn_tools

import datetime
import socket
print("Generated on %s on %s" % (datetime.date.today(), socket.gethostname()))
errors=''

Generated on 2019-09-20 on galerkin


# Kelvin-Helmholtz

We run the Kelvin-Helmholtz instability for different resolutions

Remember that 2D FVM has a runtime of $$O(N^3)$$. We can not afford to allocate that many CPUs, so we always increase the amount of CPUs by a factor of 4, and measure the time per timestep

## With file output


In [2]:
setups=['multix', 'multiy', 'multixmultiy']
build_types = []
revisions = []
min_runtime_timestep = None
max_runtime_timestep = None
for setup in setups:
    basename = '../runs/{setup}/2d/weak_scaling/kelvinhelmholtz_{N}/alsuqcli_kh_{N}_report.json'
    
    resolutions = [256, 512, 1024, 2048, 4096, 8192]
    runtimes = []
    runtimes_per_timestep = []
    core_counts = []
    for resolution in resolutions:
        try:
            report = basename.format(N=resolution, setup = setup)
            
            git_revision = alsvinn_tools.get_revision(report)
            revisions.append(git_revision)
            build_types.append(alsvinn_tools.get_build_type(report))
            runtime = alsvinn_tools.get_runtime(report)
            runtimes.append(runtime)
            
            cores = alsvinn_tools.get_total_cores(report)
            core_counts.append(cores)
            
            timesteps = alsvinn_tools.get_timesteps(report)
            runtimes_per_timestep.append(float(runtime)/timesteps)
        except Exception as e:
            errors +="Failed at resolution {N} with {setup}\n\n Skipping\n.".format(N=resolution, setup=setup)
            errors +=str(e) + '\n'
            
    try:
        if len(runtimes) == 0:
            continue
        plt.figure(0)
        plt.title("Runtime (Each core gets %d cells)" % (resolutions[0]*resolutions[0]))
        plt.loglog(core_counts, runtimes, '-o',label=setup)
        plt.xlabel('Number of cores')
        plt.ylabel('Runtime (ms)')
        ax = plt.gca()
        ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    
        
        plt.figure(1)
        plt.title("Runtime per timestep (Each core gets %d cells)" % (resolutions[0]*resolutions[0]))
        plt.loglog(core_counts, runtimes_per_timestep, '-o',label=setup)
        if min_runtime_timestep is None:
            min_runtime_timestep = min(runtimes_per_timestep)
            max_runtime_timestep = max(runtimes_per_timestep)
        else:
            min_runtime_timestep = min(min_runtime_timestep, min(runtimes_per_timestep))
            max_runtime_timestep = max(max_runtime_timestep, max(runtimes_per_timestep))
        plt.ylim([min_runtime_timestep/8, max_runtime_timestep*8])
        plt.xlabel('Number of cores')
        plt.ylabel('Runtime per timestep (ms)')
        
        ax = plt.gca()
        ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    except Exception as e:
        errors += "Failed at {setup}\n\n Skipping.".format(setup=setup) + '\n'
        errors += str(e) + '\n'
    
    print("core_counts['%s'] = %s" % (setup, core_counts))
    print("runtimes['%s'] = %s" % (setup, runtimes))
    print("runtimes_per_timestep['%s'] = %s" % (setup, runtimes_per_timestep))
    print("")
try:
    print("Git revision: %s" % git_revision )
    plt.show()
    print("revisions= %s" % revisions)
    print()
    print("build_types: %s" % build_types)
except Exception as e:
    errors+= str(e)+'\n'

## Without file output


In [3]:
setups=['multix', 'multiy', 'multixmultiy']
build_types = []
revisions = []
for setup in setups:
    basename = '../runs/{setup}/2d/weak_scaling_without_write/kelvinhelmholtz_{N}/alsuqcli_kh_{N}_report.json'
    
    resolutions = [256, 512, 1024, 2048, 4096, 8192]
    runtimes = []
    runtimes_per_timestep = []
    core_counts = []
    for resolution in resolutions:
        try:
            report = basename.format(N=resolution, setup = setup)
            
            git_revision = alsvinn_tools.get_revision(report)
            revisions.append(git_revision)
            build_types.append(alsvinn_tools.get_build_type(report))
            runtime = alsvinn_tools.get_runtime(report)
            runtimes.append(runtime)
            
            cores = alsvinn_tools.get_total_cores(report)
            core_counts.append(cores)
            
            timesteps = alsvinn_tools.get_timesteps(report)
            runtimes_per_timestep.append(float(runtime)/timesteps)
        except Exception as e:
            errors += "Failed at resolution {N} with {setup}\n\n Skipping.\n".format(N=resolution, setup=setup)
            errors += str(e) + '\n'
            
    try:
        if len(runtimes) == 0:
            continue
        plt.figure(0)
        plt.title("Runtime (Each core gets %d cells)" % (resolutions[0]*resolutions[0]))
        plt.loglog(core_counts, runtimes, '-o',label=setup)
        plt.xlabel('Number of cores')
        plt.ylabel('Runtime (ms)')
        ax = plt.gca()
        ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    
        
        plt.figure(1)
        plt.title("Runtime per timestep (Each core gets %d cells)" % (resolutions[0]*resolutions[0]))
        plt.loglog(core_counts, runtimes_per_timestep, '-o',label=setup)
        plt.xlabel('Number of cores')
        plt.ylim([min(runtimes_per_timestep)/8.0, max(runtimes_per_timestep)*8.0])
        plt.title("Runtime per timestep (Each core gets %d cells)" % (resolutions[0]*resolutions[0]))
        
        ax = plt.gca()
        ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    except Exception as e:
        errors += "Failed at {setup}\n\n Skipping.\n".format(setup=setup)
        errors += str(e) + '\n'
    
    print("core_counts['%s'] = %s" % (setup, core_counts))
    print("runtimes['%s'] = %s" % (setup, runtimes))
    print("runtimes_per_timestep['%s'] = %s" % (setup, runtimes_per_timestep))
    print("")
try:
    print("Git revision: %s" % git_revision )
    plt.show()
    print("revisions= %s" % revisions)
    print()
    print("build_types: %s" % build_types)
except Exception as e:
    errors += str(e) + '\n'

In [4]:
import os
cmakeCache = '../alsvinn/build/CMakeCache.txt'
if os.path.exists(cmakeCache):
    with open(cmakeCache, 'r') as f:
        print(f.read())

In [5]:
print(errors)

Failed at resolution 256 with multix

 Skipping
.[Errno 2] No such file or directory: '../runs/multix/2d/weak_scaling/kelvinhelmholtz_256/alsuqcli_kh_256_report.json'
Failed at resolution 512 with multix

 Skipping
.[Errno 2] No such file or directory: '../runs/multix/2d/weak_scaling/kelvinhelmholtz_512/alsuqcli_kh_512_report.json'
Failed at resolution 1024 with multix

 Skipping
.[Errno 2] No such file or directory: '../runs/multix/2d/weak_scaling/kelvinhelmholtz_1024/alsuqcli_kh_1024_report.json'
Failed at resolution 2048 with multix

 Skipping
.[Errno 2] No such file or directory: '../runs/multix/2d/weak_scaling/kelvinhelmholtz_2048/alsuqcli_kh_2048_report.json'
Failed at resolution 4096 with multix

 Skipping
.[Errno 2] No such file or directory: '../runs/multix/2d/weak_scaling/kelvinhelmholtz_4096/alsuqcli_kh_4096_report.json'
Failed at resolution 8192 with multix

 Skipping
.[Errno 2] No such file or directory: '../runs/multix/2d/weak_scaling/kelvinhelmholtz_8192/alsuqcli_kh_8192